# Calculate daily minimum temperature

Daily minimum temperature is used for the heatwave indicator.

- Estimate the minimum daily temperature from the minimum of the 6-hourly ERA-Interim data. Save this as a new dataset.
- Calculate a reference file for minimum temperatures

In [3]:
from collections import namedtuple
from pathlib import Path

import numpy as np
import pandas as pd

import xarray as xr

from config import DATA_SRC

_Row = namedtuple('_Row', ['ncdf_name', 'common_name'])

# Manually extracted from the weather files.
# python compat var long name, netcdf file name, common name
WEATHER_NAMES = [
    _Row('t2m', 'temperature_2m'),
    _Row('tp', 'precipitation'),
    _Row('d2m', 'temperature_dewpoint'),
    _Row('sp', 'surface_pressure'),
    _Row('2T_GDS4_SFC', 'temperature_2m'),
    _Row('g4_lat_1', 'latitude'),
    _Row('g4_lon_2', 'longitude'),
    _Row('initial_time0_hours', 'time')
]


def weather_mfdataset(root_path, rename=True):
    data = xr.open_mfdataset(str(root_path) + '/*.nc', engine='scipy')
    if rename:
        data = data.rename({r.ncdf_name: r.common_name for r in WEATHER_NAMES if r.ncdf_name in data.variables})
    return data





In [5]:
SUBDAILY_TEMPERATURES_FOLDER = DATA_SRC / 'weather' / 'ecmwf' / 'summer_temperature'
MIN_TEMPERATURES_FOLDER = DATA_SRC / 'weather' / 'ecmwf' / 'summer_min_temperature'

In [ ]:
# Resample the 6-hour data to daily using min, then save as a multi-file dataset to the given folder
era_alltemps = weather_mfdataset(SUBDAILY_TEMPERATURES_FOLDER)
era_allmin = era_alltemps.resample(time='1D').min()

years, datasets = zip(*era_allmin.groupby('time.year'))
paths = [MIN_TEMPERATURES_FOLDER / f'{y}_daily_min_temperature.nc' for y in years]
xr.save_mfdataset(datasets, paths)

## Calculate the climatology/refrence using the base period 1986-2005

In [9]:
allmin_reference = era_allmin.sel(time=slice('1986-01-01', '2005-12-31')).load()
# high_t_reference = high_t_reference.quantile(0.99, dim='time')

In [6]:
era_allmin = xr.open_mfdataset(str(MIN_TEMPERATURES_FOLDER / '*.nc'))

In [22]:
CLIMATOLOGY_DAILY_FILE = DATA_SRC / 'weather' / 'ecmwf' / 'climatology_daily_tmin_quantiles.nc'

In [20]:
# TODO maybe add more quantiles
tmin_quantiles = allmin_reference.quantile([0.25, 0.5, 0.75, 0.99], dim='time')
tmin_quantiles.to_netcdf(CLIMATOLOGY_DAILY_FILE)